In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
apple_sentiment = pd.read_csv("sentiment_time_series/apple_sentiment_time_series.csv")
apple_sentiment.head()

,date,binary_mean,binary_squared_mean,binary_sqrt_mean,prob_mean,prob_squared_mean,prob_sqrt_mean
0,1/2/2020,0.0,0.0,0.0,0.0,0.0,0.0
1,1/3/2020,0.0,0.0,0.0,0.0,0.0,0.0
2,1/4/2020,0.0,0.0,0.0,0.0,0.0,0.0
3,1/5/2020,0.0,0.0,0.0,0.0,0.0,0.0
4,1/6/2020,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
apple_lstm = pd.read_csv("AAPL_LSTM_predictions.csv")
apple_lstm.head()

,Unnamed: 0,Date,Pred,Actual
0,7559,1/2/2020,71.986710,74.444603
1,7560,1/3/2020,73.601325,73.720840
2,7561,1/6/2020,72.764445,74.308266
3,7562,1/7/2020,73.520607,73.958794
4,7563,1/8/2020,73.079492,75.148521


In [4]:
merged = pd.merge(apple_sentiment, apple_lstm, left_on='date', right_on='Date')

In [5]:
merged_changed = merged[['date', 'prob_sqrt_mean', 'Pred', 'Actual']]
merged_changed = merged_changed.dropna()

In [6]:
inputs_1 = merged_changed.prob_sqrt_mean.values
inputs_1.shape

(155,)

In [7]:
inputs_2 = merged_changed.Pred.values
inputs_2.shape

(155,)

In [8]:
inputs = np.stack((inputs_1, inputs_2), axis=1)
inputs.shape

(155, 2)

In [9]:
targets = merged_changed.iloc[:, 3].values

In [10]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
inputs = sc.fit_transform(inputs)

inputs = sm.add_constant(inputs)
model = sm.OLS(targets, inputs)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                     4177.
Date:                Fri, 12 Feb 2021   Prob (F-statistic):          1.45e-133
Time:                        22:10:43   Log-Likelihood:                -364.68
No. Observations:                 155   AIC:                             735.4
Df Residuals:                     152   BIC:                             744.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         90.2209      0.206    437.198      0.0

In [11]:
predictions = results.predict()

In [12]:
apple_stock_prediction = pd.DataFrame().assign(Predictions=predictions, 
                                                Actual=merged['Actual'])
apple_stock_prediction.to_csv("apple_stock_prediction_reddit.csv")